In [1]:
# !sudo sh -c 'apt-get update && apt-get upgrade && apt-get install cmake'
# !pip3 install https://github.com/kpu/kenlm/archive/master.zip
# !pip3 install levenshtein

In [2]:
# !git clone https://github.com/AntonOkhotnikov/ai-talent-hub-itmo-speech-course.git
# !mv ai-talent-hub-itmo-speech-course/assignments/assignment2/* .

In [ ]:
!wget https://www.openslr.org/resources/11/3-gram.arpa.gz -P lm
!wget https://www.openslr.org/resources/11/3-gram.pruned.3e-7.arpa.gz -P lm
!wget https://www.openslr.org/resources/11/4-gram.arpa.gz -P lm

In [4]:
!pip3 freeze | grep -E "kenlm|torch*|Levenshtein|transformers|numpy|pandas"

geopandas==1.0.1
kenlm @ https://github.com/kpu/kenlm/archive/master.zip#sha256=f935c2ac0be6dd1bf96a5852973dad6103c48e866cf33532459fb1336bea4bfe
Levenshtein==0.27.1
numpy==2.0.2
pandas==2.2.2
pandas-datareader==0.10.0
pandas-gbq==0.28.0
pandas-stubs==2.2.2.240909
sentence-transformers==3.4.1
sklearn-pandas==2.2.0
torch @ https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchsummary==1.5.1
torchvision @ https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl
transformers==4.50.3


In [5]:
import math
import time
import heapq
from pathlib import Path
from typing import List, Tuple, Union

import kenlm
import torch
import torchaudio
import Levenshtein
import numpy as np
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [6]:
class Wav2Vec2Decoder:
    def __init__(
            self,
            model_name="facebook/wav2vec2-base-960h",
            lm_model_path="lm/3-gram.pruned.1e-7.arpa.gz",
            beam_width=3,
            alpha=1.0,
            beta=1.0
        ):
        self.processor = Wav2Vec2Processor.from_pretrained(model_name)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_name)

        self.vocab = {i: c for c, i in self.processor.tokenizer.get_vocab().items()}
        self.blank_token_id = self.processor.tokenizer.pad_token_id
        self.word_delimiter = self.processor.tokenizer.word_delimiter_token
        self.beam_width = beam_width
        self.alpha = alpha
        self.beta = beta
        self.lm_model = kenlm.Model(lm_model_path) if lm_model_path else None

    def greedy_decode(self, logits: torch.Tensor) -> str:
        pred_idxs = torch.argmax(logits, dim=-1)
        return self._seq_to_transcript(pred_idxs.tolist())

    def _seq_to_transcript(self, seq):
        tokens = []
        prev_token = None
        for idx in seq:
            if idx != self.blank_token_id and idx != prev_token:
                tokens.append(self.vocab[idx])
            prev_token = idx
        return ''.join(tokens).replace(self.word_delimiter, ' ').strip()

    def beam_search_decode(self, logits: torch.Tensor, return_beams: bool = False):
        log_probs = F.log_softmax(logits, dim=-1)
        T, V = log_probs.shape

        beams = [([], 0.0)]  # (sequence, score)

        for t in range(T):
            new_beams = []
            for seq, score in beams:
                for v in range(V):
                    new_seq = seq + [v]
                    new_score = score + log_probs[t, v].item()
                    new_beams.append((new_seq, new_score))
            beams = heapq.nlargest(self.beam_width, new_beams, key=lambda x: x[1])

        if return_beams:
            return beams
        else:
            return self._seq_to_transcript(beams[0][0])

    def beam_search_with_lm(self, logits: torch.Tensor) -> str:
        log_probs = F.log_softmax(logits, dim=-1)
        T, V = log_probs.shape

        beams = [([], 0.0, "<s>")]  # (sequence, score, LM state)

        for t in range(T):
            new_beams = []
            for seq, score, lm_prefix in beams:
                for v in range(V):
                    new_seq = seq + [v]
                    token = self.vocab[v]
                    new_score = score + log_probs[t, v].item()

                    # Apply LM shallow fusion
                    if token != self.word_delimiter and token != self.processor.tokenizer.pad_token:
                        lm_state = lm_prefix + token
                        lm_score = self.lm_model.score(lm_state, bos=True, eos=False)
                        new_score += self.alpha * lm_score + self.beta
                        new_beams.append((new_seq, new_score, lm_state))
                    else:
                        new_beams.append((new_seq, new_score, lm_prefix))

            beams = heapq.nlargest(self.beam_width, new_beams, key=lambda x: x[1])

        return self._seq_to_transcript(beams[0][0])

    def lm_rescore(self, beams: List[Tuple[str, float]]) -> str:
        rescored = []
        for seq, score in beams:
            transcript = self._seq_to_transcript(seq)
            words = transcript.split()
            lm_score = self.lm_model.score(" ".join(words), bos=True, eos=True)
            total_score = self.alpha * lm_score + self.beta * len(words) + score
            rescored.append((transcript, total_score))
        rescored.sort(key=lambda x: x[1], reverse=True)
        return rescored[0][0]

    def decode(self, audio_input: torch.Tensor, method: str = "greedy") -> str:
        inputs = self.processor(audio_input, return_tensors="pt", sampling_rate=16000)
        with torch.no_grad():
            logits = self.model(inputs.input_values.squeeze(0)).logits[0]

        if method == "greedy":
            return self.greedy_decode(logits)
        elif method == "beam":
            return self.beam_search_decode(logits)
        elif method == "beam_lm":
            return self.beam_search_with_lm(logits)
        elif method == "beam_lm_rescore":
            beams = self.beam_search_decode(logits, return_beams=True)
            return self.lm_rescore(beams)
        else:
            raise ValueError("Invalid decoding method. Choose one of 'greedy', 'beam', 'beam_lm', 'beam_lm_rescore'.")


def test(decoder, audio_path, true_transcription):

    res_dict = {}
    audio_input, sr = torchaudio.load(audio_path)
    assert sr == 16000, "Audio sample rate must be 16kHz"

    print("=" * 60)
    print("Target transcription")
    print(true_transcription)

    # Print all decoding methods results
    for d_strategy in ["greedy", "beam", "beam_lm", "beam_lm_rescore"]:
        print("-" * 60)
        print(f"{d_strategy} decoding")
        t0 = time.perf_counter()
        transcript = decoder.decode(audio_input, method=d_strategy).strip()
        res_dict[f'{d_strategy}_t'] = time.perf_counter() - t0
        print(f"{transcript}")
        score = Levenshtein.distance(true_transcription, transcript) / max(len(true_transcription), len(transcript))
        print(f"Character-level Levenshtein distance: {score}")
        res_dict[f'{d_strategy}_levenshtein'] = score
    return res_dict

In [7]:
  test_samples = [
      ("examples/sample1.wav", "IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL READY TO ACKNOWLEDGE MYSELF YOUR DEBTOR FOR AN ACT OF THE MOST NOBLE FORBEARANCE"),
      ("examples/sample2.wav", "AND IF ANY OF THE OTHER COPS HAD PRIVATE RACKETS OF THEIR OWN IZZY WAS UNDOUBTEDLY THE MAN TO FIND IT OUT AND USE THE INFORMATION WITH A BEAT SUCH AS THAT EVEN GOING HALVES AND WITH ALL THE GRAFT TO THE UPPER BRACKETS HE'D STILL BE ABLE TO MAKE HIS PILE IN A MATTER OF MONTHS"),
      ("examples/sample3.wav", "GUESS A MAN GETS USED TO ANYTHING HELL MAYBE I CAN HIRE SOME BUMS TO SIT AROUND AND WHOOP IT UP WHEN THE SHIPS COME IN AND BILL THIS AS A REAL OLD MARTIAN DEN OF SIN"),
      ("examples/sample4.wav", "IT WAS A TUNE THEY HAD ALL HEARD HUNDREDS OF TIMES SO THERE WAS NO DIFFICULTY IN TURNING OUT A PASSABLE IMITATION OF IT TO THE IMPROVISED STRAINS OF I DIDN'T WANT TO DO IT THE PRISONER STRODE FORTH TO FREEDOM"),
      ("examples/sample5.wav", "MARGUERITE TIRED OUT WITH THIS LONG CONFESSION THREW HERSELF BACK ON THE SOFA AND TO STIFLE A SLIGHT COUGH PUT UP HER HANDKERCHIEF TO HER LIPS AND FROM THAT TO HER EYES"),
      ("examples/sample6.wav", "AT THIS TIME ALL PARTICIPANTS ARE IN A LISTEN ONLY MODE"),
      ("examples/sample7.wav", "THE INCREASE WAS MAINLY ATTRIBUTABLE TO THE NET INCREASE IN THE AVERAGE SIZE OF OUR FLEETS"),
      ("examples/sample8.wav", "OPERATING SURPLUS IS A NON CAP FINANCIAL MEASURE WHICH IS DEFINED AS FULLY IN OUR PRESS RELEASE"),
  ]


In [8]:
def score(decoder, audio_input, true_transcription, d_strategies, lm_model_name=None):
    res_records = []
    try:
        for d_strategy in d_strategies:
            res_dict = {'method': d_strategy, 'beam_width': decoder.beam_width,
                        'alpha': decoder.alpha, 'beta': decoder.beta,
                        'lm_model': lm_model_name}
            t0 = time.perf_counter()
            transcript = decoder.decode(audio_input, method=d_strategy)
            res_dict[f't'] = time.perf_counter() - t0
            res_dict[f'transcript'] = transcript
            score = Levenshtein.distance(true_transcription, transcript.strip())  / max(len(true_transcription), len(transcript))
            res_dict[f'levenshtein'] = score
            res_records.append(res_dict)
        print(res_records)
    except Exception as e:
        print(e)
    finally:
        return res_records

In [ ]:
import pandas as pd

In [9]:
model_paths = list(filter(lambda x: x.is_file(), Path('lm').glob('*')))
beam_width_grid = np.linspace(1, 10, 5, dtype=int)
ratio_grid = np.linspace(0.1, 1, 5, dtype=float)
scale_grid = np.linspace(1, 5, 5, dtype=int)
model_paths, beam_width_grid, ratio_grid, scale_grid

([PosixPath('lm/4-gram.arpa.gz'),
  PosixPath('lm/3-gram.pruned.1e-7.arpa.gz'),
  PosixPath('lm/3-gram.pruned.3e-7.arpa.gz'),
  PosixPath('lm/3-gram.arpa.gz')],
 array([ 1,  3,  5,  7, 10]),
 array([0.1  , 0.325, 0.55 , 0.775, 1.   ]),
 array([1, 2, 3, 4, 5]))

In [ ]:
score_records = []
pbar = tqdm(total=
            len(model_paths) * len(test_samples) + \
            len(model_paths) * len(test_samples) * (len(beam_width_grid) + 1) + \
            len(model_paths) * len(test_samples) * (len(beam_width_grid) + 1) * (len(ratio_grid) + 1) * len(scale_grid) * 2
            )
for model_path in model_paths:
    decoder = Wav2Vec2Decoder(lm_model_path=str(model_path), beam_width=1, alpha=0.1, beta=0.9)
    for audio_path, target in test_samples:
        audio_input, sr = torchaudio.load(audio_path)

        score_records.extend(score(decoder, audio_input, target, ["greedy", "beam", "beam_lm", "beam_lm_rescore"], model_path.stem))
        pbar.update(4)
        for beam_width in beam_width_grid[1:]:
            decoder.beam_width = int(beam_width)
            score_records.extend(score(decoder, audio_input, target, ['beam', "beam_lm", "beam_lm_rescore"], model_path.stem))
            pbar.update(3)
            for ratio in ratio_grid[1:]:
                for scale in scale_grid[1:]:
                    decoder.alpha = round(ratio * scale, 1)
                    decoder.beta = round(scale - decoder.alpha, 1)
                    score_records.extend(score(decoder, audio_input, target, ["beam_lm", "beam_lm_rescore"], model_path.stem))
                    pbar.update(2)


  0%|          | 0/11744 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 4/11744 [02:47<136:14:52, 41.78s/it]

[{'method': 'greedy', 'beam_width': 1, 'alpha': 0.1, 'beta': 0.9, 'lm_model': '4-gram.arpa', 't': 4.258842862000165, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO ACKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.03902439024390244}, {'method': 'beam', 'beam_width': 1, 'alpha': 0.1, 'beta': 0.9, 'lm_model': '4-gram.arpa', 't': 5.452081368999643, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO ACKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.03902439024390244}, {'method': 'beam_lm', 'beam_width': 1, 'alpha': 0.1, 'beta': 0.9, 'lm_model': '4-gram.arpa', 't': 4.385218063998764, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL 

  0%|          | 7/11744 [03:03<74:49:07, 22.95s/it] 

[{'method': 'beam', 'beam_width': 3, 'alpha': 0.1, 'beta': 0.9, 'lm_model': '4-gram.arpa', 't': 5.271629992999806, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO ACKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.03902439024390244}, {'method': 'beam_lm', 'beam_width': 3, 'alpha': 0.1, 'beta': 0.9, 'lm_model': '4-gram.arpa', 't': 5.010692875999666, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETHER TO ACKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.04390243902439024}, {'method': 'beam_lm_rescore', 'beam_width': 3, 'alpha': 0.1, 'beta': 0.9, 'lm_model': '4-gram.arpa', 't': 5.833143184998335, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I Y

  0%|          | 9/11744 [03:14<55:56:58, 17.16s/it]

...

[{'method': 'beam_lm', 'beam_width': 7, 'alpha': np.float64(1.3), 'beta': np.float64(2.7), 'lm_model': '4-gram.arpa', 't': 7.1792875950013695, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO CKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.04390243902439024}, {'method': 'beam_lm_rescore', 'beam_width': 7, 'alpha': np.float64(1.3), 'beta': np.float64(2.7), 'lm_model': '4-gram.arpa', 't': 5.793245403001492, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO ACKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.03902439024390244}]


  1%|          | 85/11744 [10:44<20:59:06,  6.48s/it]

[{'method': 'beam_lm', 'beam_width': 7, 'alpha': np.float64(1.6), 'beta': np.float64(3.4), 'lm_model': '4-gram.arpa', 't': 7.074567448999005, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO CKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.04390243902439024}, {'method': 'beam_lm_rescore', 'beam_width': 7, 'alpha': np.float64(1.6), 'beta': np.float64(3.4), 'lm_model': '4-gram.arpa', 't': 6.151381325998955, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO ACKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.03902439024390244}]


  1%|          | 87/11744 [10:57<21:04:27,  6.51s/it]

[{'method': 'beam_lm', 'beam_width': 7, 'alpha': np.float64(1.1), 'beta': np.float64(0.9), 'lm_model': '4-gram.arpa', 't': 6.885525534999033, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETE TO CKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.04390243902439024}, {'method': 'beam_lm_rescore', 'beam_width': 7, 'alpha': np.float64(1.1), 'beta': np.float64(0.9), 'lm_model': '4-gram.arpa', 't': 6.2678978980002285, 'transcript': 'IF YOU ARE GENEROUS HERE IS A FITTING OPPORTUNITY FOR THE EXERCISE OF YOUR MAGNANIMITY IF YOU ARE PROUD HERE AM I YOUR RIVAL RETER TO ACKNOWLEDGE MYSELF YOUR DEPTOR FOR AN ACT OF MOST NOBLE FORBEARANCE', 'levenshtein': 0.03902439024390244}]


In [ ]:
[test(decoder, audio_path, target) for audio_path, target in test_samples]

In [ ]:
scores_df = pd.DataFrame.from_records([test(decoder, audio_path, target) for audio_path, target in test_samples])
scores_df